In [6]:
%pylab notebook

import os, sys, pandas as pd
sys.path.append('/Users/alexisdevitre/Documents/code/hts-irradiation/')

import hts_fitting as hts
import hts_showcase as sc

def get_fluence_steps(fpath):
    return pd.read_excel(fpath, sheet_name='fluence_steps', usecols=[0, 1, 2], names=['step', 'step_fluence', 'cumulated_fluence'])


%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [7]:
tape = 'ffj33'
path = '../../fluence/'
fpath = [path+f for f in os.listdir(path) if '.DS_Store' not in f][0]

fluences = get_fluence_steps(fpath)

steps = [str(f) for f in np.sort(os.listdir()) if 'step' in f]

data = None
for step in steps:
    df = hts.getIcT([step+'/'+ f for f in np.sort(os.listdir(step)) if 'Ic_' in f])[2]
    df['cumulated_fluence'] = fluences.cumulated_fluence.loc[fluences.step == step].values[0]
    if data is not None:
        data = pd.concat([data, df], ignore_index=True)
    else:
        data = df

with pd.ExcelWriter('{}-degradation.xlsx'.format(tape), mode='w', engine='openpyxl') as writer:
    data.to_excel(writer, sheet_name='all_data', index=False)
    
bins = np.arange(0.1, 93.6, 0.2)

# Create a new column for age groups
data['temperature_group'] = pd.cut(data['temperature'], bins=bins, right=False)
data = data.drop('fpath', axis=1).groupby(['temperature_group', 'cumulated_fluence']).agg(['mean', 'std', 'count']).dropna()
data.columns = ['_'.join(col) for col in data.columns]
data.drop(['temperature_count', 'ic_count', 'n_count', 'timeends_mean', 'timeends_std'], axis=1, inplace=True)
data.rename(columns={'timeends_count': 'file_count'}, inplace=True)

with pd.ExcelWriter('{}-degradation.xlsx'.format(tape), mode='a', engine='openpyxl', if_sheet_exists='new') as writer:
    data.reset_index().to_excel(writer, sheet_name='aggregates', index=False)

FileNotFoundError: [Errno 2] No such file or directory: '../../fluence/'